# Density matrix exponentiation

This is a function to simulate DME. It follows the following protocol:
![image.png](attachment:image.png)
Goals: implement the unitary $U=e^{-i\rho\theta}$ on data qubit $\sigma$ (rotate $\sigma$ by $\theta$) according to instruction given by instruction qubit $\rho$.

### Function `swap_rotation`
First, DME uses the relation (rotate $\sigma$ by small angle $\delta = \sigma/N$).
$$\begin{align}
Tr_\rho [e^{-iSWAP\delta}\sigma\otimes\rho e^{iSWAP\delta}] &= \sigma - i\delta[\rho,\sigma] +\mathcal{O}(\delta^2)\\
&= e^{-i\rho\delta}\sigma e^{i\rho\delta}+\mathcal{O}(\delta^2)
\end{align}$$


### Function `QME`
Second, DME uses either a simple reset, or QME to dynamically reset the instruction qubit.

QME apply $\mathbb{1}$, or $\pi$-rotation in the instruction state basis $R_\nu(\pi)$ with probability $p = 0.5$

Outcomes of multiple such randomized instantiations are averaged $\to$ mimic single circuit with active reset of $\rho$

### Function `DME_evolution`
Combine the above, this function execute the above circuit (without QME) or the following circuit (with QME):

![image-2.png](attachment:image-2.png)


In [167]:
import numpy as np
from qibo.config import raise_error
from abc import abstractmethod

import qibo
from qibo import gates, models
from enum import Enum, auto

def test_qdp_memory_usage_query_circuit():
    qdp = quantum_dynamic_programming(1,10)
    qibo.set_backend("qibolab", platform="my_platform")

class QDP_memory_type(Enum):
    default = auto()
    reset = auto()
    quantum_measurement_emulation = auto()

class quantum_dynamic_programming():
    def __init__(self, num_target_qubits, num_instruction_qubits, number_muq_per_call):
        self.num_target_qubits = int(num_target_qubits)
        self.num_instruction_qubits = int(num_instruction_qubits)
        #SAM: fix id on each qubit, if workin put in work_reg, otherwise instruction_reg
        self.list_id_target_reg = np.arange(0,num_target_qubits,1)
        self.list_id_instruction_reg = np.arange(0,num_instruction_qubits,1) + num_target_qubits
        #SAM: since instruction can be multi qubit, id_current_instruction_reg should be list/arr
        self.id_current_instruction_reg = self.list_id_instruction_reg[0]
        self.M = number_muq_per_call
        self.memory_type = QDP_memory_type.default
        self.c = models.Circuit(self.num_target_qubits+ self.num_instruction_qubits)
    
    def __call__(self):
        # return entire circuit
        return self.memory_call_circuit()
        # for now we assume that the recursion step make only 1 type memory call per QDP step

    
    def memory_call_circuit(self, num_instruction_qubits_per_query):
        # return the entire circuit
        if self.memory_type == QDP_memory_type.default:
            self.list_id_current_instruction_reg = self.list_id_instruction_reg[self.id_current_instruction_reg:self.M*num_instruction_qubits_per_query+self.id_current_instruction_reg]-1
            self.instruction_qubits_initialization()
            for register in self.list_id_current_instruction_reg:
                self.memory_usage_query_circuit()
                self.trace_instruction_qubit()
                self.increment_current_instruction_register()
        
        elif self.memory_type == QDP_memory_type.reset:
            for _register_use in range(self.M):
                self.memory_usage_query_circuit()
                self.trace_instruction_qubit()
                self.single_register_reset(self.id_current_instruction_reg)
                
        elif self.memory_type == QDP_memory_type.quantum_measurement_emulation:
            for _register_use in range(self.M):
                self.memory_usage_query_circuit()
                self.trace_instruction_qubit()
                self.QME(self.id_current_instruction_reg)

    def single_register_reset(self,register):
        self.c.add(gates.reset(register))

    def all_register_reset(self):
        for qubit in self.num_instruction_qubits:
            self.c.add(gates.reset(qubit))
        self.id_current_instruction_reg = 0

    def circuit_reset(self):
        self.c = models.Circuit(self.num_target_qubits + self.num_instruction_qubits)
    
    def increment_current_instruction_register(self):
        self.id_current_instruction_reg += 1
      
    @abstractmethod
    def memory_usage_query_circuit(self, N, id_target_reg, id_instruction_reg):
        # N is the gate
        #todo: ask them how to change gates duration
        #self.c.add(gates.N(id_target_reg,id_instruction_reg))
        raise_error(NotImplementedError)
    
    def instruction_qubits_initialization(self):
        pass
    
    def QME(self,register):
        import random
        coin_flip = random.choice([0, 1])
        if coin_flip == 0:
            QME_gate = gates.R_nu(np.pi,register) # nu is the vector parallel to rho
        elif coin_flip == 1:
            QME_gate = gates.identity(register)
        self.c.add(QME_gate)

    def trace_instruction_qubit(self):
        for qubit in self.list_id_current_instruction_reg:
            self.c.add(gates.M(qubit))

In [168]:
import math
from typing import List

import numpy as np

from qibo.config import PRECISION_TOL, raise_error
from qibo.gates.abstract import Gate, ParametrizedGate
from qibo.parameter import Parameter

class delta_SWAP(Gate):
    """The swap gate.

    Corresponds to the following unitary matrix

    .. math::
        \\begin{pmatrix}
        1 & 0 & 0 & 0 \\\\
        0 & 0 & 1 & 0 \\\\
        0 & 1 & 0 & 0 \\\\
        0 & 0 & 0 & 1 \\\\
        \\end{pmatrix}

    Args:
        q0 (int): the first qubit to be swapped id number.
        q1 (int): the second qubit to be swapped id number.
    """

    def __init__(self, q0, q1):
        super().__init__()
        self.name = "delta_swap"
        self.draw_label = "d_x"
        self.target_qubits = (q0, q1)
        self.init_args = [q0, q1]
        self.unitary = True

    @property
    def clifford(self):
        return False

    @property
    def qasm_label(self):
        return "delta_swap"

In [169]:
class density_matrix_exponentiation(quantum_dynamic_programming):
    def __init__(self, theta, N, num_target_qubits, num_instruction_qubits, number_muq_per_call):
        super().__init__(num_target_qubits, num_instruction_qubits, number_muq_per_call)
        self.theta = theta # overall rotation angle
        self.N = N # number of steps
        self.delta = theta/N # small rotation angle
        self.memory_type = QDP_memory_type.default
        self.id_curent_target_reg = self.list_id_target_reg[0]

    def memory_usage_query_circuit(self):
        self.c.add(gates.SWAP(self.id_curent_target_reg,self.id_current_instruction_reg))

    def instruction_qubits_initialization(self):
        for instruction_qubit in self.list_id_current_instruction_reg:
            self.c.add(gates.X(instruction_qubit))

In [172]:
test_1 = density_matrix_exponentiation(theta=np.pi,N=1,num_target_qubits=1,num_instruction_qubits=6,number_muq_per_call=1)
test_1.memory_call_circuit(3)

In [173]:
print('DME, q0 is target qubit, q1,q2 and q3 are instruction qubit')
print(test_1.c.draw())

DME, q0 is target qubit, q1,q2 and q3 are instruction qubit
q0: ───x───x───x───
q1: ─X─x─M─|─M─|─M─
q2: ─X───M─x─M─|─M─
q3: ─X───M───M─x─M─
q4: ───────────────
q5: ───────────────
q6: ───────────────


In [115]:
result = test_1.c.execute()

In [2]:
def partial_trace(dm, sub_index):
    """
    Calculate the partial trace over the second qubit.

    Parameters:
    dm : numpy.ndarray
        The 4x4 density matrix of a two-qubit system.
    sub_index : int
        The index of the subsystem to trace out. 1 for tracing out the first qubit,
        and 2 for tracing out the second qubit.

    Returns:
    numpy.ndarray
        The 2x2 reduced density matrix of the specified qubit.
    """
    # Reshape dm from a 4x4 matrix into a 2x2x2x2 tensor
    reshaped_dm = dm.reshape(2, 2, 2, 2)

    # Perform the partial trace by summing out the inner 2x2 matrix
    if sub_index not in [1, 2]:
        raise ValueError('Sub index should be in range [1, 2]')

    if sub_index == 1:
        # Trace out the second subsystem
        reduced_dm = np.einsum('ijik->jk', reshaped_dm)

    if sub_index == 2:
        # Trace out the first subsystem
        reduced_dm = np.einsum('jiki->jk', reshaped_dm)

    return reduced_dm

In [3]:
class quantum_dynamic_programming_numpy:
    # SAM: matrix representatio of QDP call (circuit)
    def __init__(self, sigma):
        self.sigma = sigma

    @abstractmethod
    def memory_usage_query(self,N_op,rho,sigma = None):
        if sigma == None:
            sigma = self.sigma
        return trace_1(np.exp(-1j*N_op) * tensor(rho,sigma) * np.exp(1j*N_op))
        
    #todo look up qibo evolution

    def memory_call(self):
        raise_error(NotImplementedError)



class density_matrix_exponentiation(quantum_dynamic_programming):
    def __init__(self, rho, sigma, theta, N):
        self.rho = rho # instruction qubit
        self.sigma = sigma # data_qubit
        self.theta = theta # overall rotation angle
        self.N = N # number of steps
        self.delta = theta/N # small rotation angle
        self.partial_swap_negative = np.exp(-1j*swap()*self.delta)
        self.partial_swap_positive = np.exp(1j*swap()*self.delta)

    def DME_evolution(self):
        current_sigma, current_rho = self.sigma, self.rho
        for _ in range(self.N):
            current_sigma, current_rho = self.swap_rotation(current_sigma, current_rho)
        return current_sigma, current_rho

    def swap_rotation(self, current_sigma, current_rho, QME = False):        
        system = tensor(current_sigma, current_rho)
        unitary_evolution = self.partial_swap_negative * system * self.partial_swap_positive
        new_sigma = trace_rho(unitary_evolution)
        if QME == True:
            new_rho = self.QME(current_rho)
        elif QME == False:
            new_rho = self.rho 
        return new_sigma, new_rho

    def QME(self, current_rho):
        import random
        coin_flip = random.choice([0, 1])
        if coin_flip == 0:
            QME_gate = R_nu(np.pi) # nu is the vector parallel to rho
        elif coin_flip == 1:
            QME_gate = identity()
        new_rho = QME_gate * current_rho
        return new_rho

    def measurement(self):
        # measurement or state tomography
        current_sigma, current_rho = self.DME_evolution()
        final_sigma = measurement_gate * current_sigma
        final_rho = measurement_gate * current_rho
        return final_sigma, final_rho